# 03 - Redes Neurais Recorrentes - LSTM - Previsão a Cada Passo

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [3]:
data = pd.read_csv('data-processed/ts_hr_feb_2020.csv', parse_dates=['date'])

data['hour'] = data['date'].dt.hour

last_sunday = pd.to_datetime("2020-02-23")
train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [4]:
def prep_seqs(df, l=24, h=24):
    X = []
    Y = []
    for i in range(l,df.shape[0]-h):
        f = df.iloc[i-l:i][['sales_value', 'hour']] #hoje
        y = df.iloc[i:i+h]['sales_value'] # 1 dia depois

        X.append(f.values)
        Y.append(y.values)

    X = np.array(X)
    Y = np.array(Y)

    #X = np.expand_dims(X, -1)
    X = np.swapaxes(X, 0,1)
    
    #Y = np.expand_dims(Y, -1)
    #Y = np.swapaxes(Y, 0,2)
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [5]:
X, Y = prep_seqs(train)
Xt, Yt = prep_seqs(val)

mean_train = X.mean(1, keepdims=True)
mean_train[:, :, 1] = 0
std_train = X.std(1,  unbiased=False, keepdims=True)
std_train[:, :, 1] = 1

meanY_train = Y.mean(0, keepdims=True)
stdY_train = Y.std(0, unbiased=False, keepdims=True)

X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train


Y -= meanY_train
Y /= stdY_train


In [6]:
class LSTMRNN(nn.Module):
    def __init__(self, hidden_size):
        super(LSTMRNN, self).__init__()
        self.hidden_size = hidden_size
        self.hour_emb = nn.Embedding(24, 4)
        self.lstm = nn.LSTM(5, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, 1)
        
    def forward(self, input):
        #print("Inputs", input.shape)
        hour_emb = self.hour_emb(input[:,:,-1].long())
        nums = input[:,:,:-1]
        
        inputs = torch.cat([nums, hour_emb], dim=2)

        output, _ = self.lstm(inputs)
        #print("Output LSTM", output.shape)
        
        output = self.out(output) 
        #print("Output do self.out", output.shape)
        output = output.squeeze()
        #print("Depois do squeeze", output.shape)
        output = torch.transpose(output, 0, 1) 
        #print("Depois do transpose", output.shape)
        return output
    

In [7]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

lstm = LSTMRNN(100)
print(lstm)

LSTMRNN(
  (hour_emb): Embedding(24, 4)
  (lstm): LSTM(5, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [8]:
from sklearn.metrics import mean_absolute_error

criterion = nn.L1Loss()
lstm_optimizer = optim.Adam(lstm.parameters(), lr=1e-2)

In [10]:
for i in range(100):
    lstm.zero_grad()

    o = lstm(X)
    loss = criterion(o, Y)
    loss.backward()
    lstm_optimizer.step()
    #print(loss.item())
    
    
    p = lstm(Xt)
    p = p.detach()
    
    p *= stdY_train
    p += meanY_train
    p = p.numpy().squeeze()
    
    
print(mean_absolute_error(Yt.numpy(), p))

360.07095


# Fim